In [1]:
import pandas as pd

In [2]:
#импортировали библиотеки
from datetime import datetime
from decimal import Decimal, getcontext

In [3]:
# сделали константу для формата времени
DATE_FORMAT = '%Y-%m-%d'
# создали словарь
goods = {}

Функция add добавляет новый продукт в список items.

Возвращает новый элемент продукта с указанным объемом amount и датой истечения срока годности expiration_date (если указана). Если продукт с таким названием уже существует в списке, то просто добавляет новое количество для продукта.

Обработка даты за счет datetime.strptime и date позволяет корректно определить формат даты в дате истечения срока годности (если был передан параметр expiration_date).

In [4]:

def add(items, title, amount, expiration_date=None):
    if expiration_date is not None: # если данные в формате : add(goods, 'Яйца', Decimal('10'), '2023-9-30')
        exp_format = datetime.strptime(expiration_date, DATE_FORMAT) # сначала парсим строку expiration_date в объект datetime с помощью метода strptime,
        exp_date = date(exp_format.year, exp_format.month, exp_format.day) # а затем вызываем метод date() для получения только объекта date.
    else: # если то же самое, НО без даты
        exp_date = None

    if title not in items: # делаем формат вывода по условию: если товара 'Яйца' еще нет в списке
        items[title] = [{'amount':amount, 'expiration_date': exp_date}]
    else: # если они там есть
        items[title].append({'amount': amount, 'expiration_date': exp_date})




Функция add_by_note упрощает добавление нового продукта в список items на основе строки note.

Разбивает строку по пробелам для извлечения названия продукта, объема и даты истечения срока годности (если передано).


После извлечения параметров title, amount и expiration_date функция вызывает функцию add с полученными значениями.

In [5]:
def add_by_note(items, note): # для add_by_note(goods, 'Яйца гусиные 4 2023-07-15')
    note_split = note.split() # делим строку ['Яйца', 'гусиные', '4', '2023-07-15']
    if '-' in note: # если в дате есть -, а оно есть, то
        expiration_date = note_split[-1] #дата - первый отрезок с конца
        amount = Decimal(note_split[-2]) #количество - второй отрезок с конца
        title = ' '.join(note_split[0:-2]) # все остальное - название продукта
    else:
        expiration_date = None
        amount = Decimal(note_split[-1])
        title = ' '.join(note_split[0:-1])
    add(items, title, amount, expiration_date)


In [6]:
string = 'Яйца гусиные 4 2023-07-15'
spl = string.split()
spl

['Яйца', 'гусиные', '4', '2023-07-15']

Функция find выполняет поиск в списке items по названию элемента (продукта) needle (в тч части слова), игнорируя регистр. Возвращает список всех найденных элементов

In [7]:
def find(items, needle):
    res = [] #создали пустой список
    for title in items: # ищем что нам там надо в items
        if needle.lower() in title.lower(): # с игнором регистра
            res.append(title) # все, что подошло, запихиваем в res
    return res


Функция amount ищет и возвращает общее значение объема для всех элементов в списке items, которые входят в результат поиска функцией find по элементу needle.

In [8]:

def amount(items, needle):
    found_items = find(items, needle) # чтобы при суммировании учитывались  только товары, содержащие в названии значение needle сохраняем эту историю в доппеременную
    value = Decimal('0')
    for title in found_items:
        for item in items[title]:
            value += item['amount']
    return value


Функция expire фильтрует элементы списка items по дате истечения срока годности (в пределах in_advance_days).

Возвращается список элементов с датой истечения срока годности, которые менее или равны текущей даты, и их количество (указанное в expired_sum).

In [9]:
def expire(items, in_advance_days=0):
    result = []
    expiration_date = date.today() + timedelta(days=in_advance_days)
    for title in items:
        expired_sum = Decimal('0')  # Перемещение инициализации переменной expired_sum, чтобы она обнулялась перед подсчетом суммы просроченных продуктов каждого тайтла
        for item in items[title]:
            if item['expiration_date'] is not None and item['expiration_date'] <= expiration_date:
                expired_sum += item['amount']
        if expired_sum > 0:
            result.append((title, expired_sum))
    return result